# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("../python-api-challenge/cities.csv")
cities_pd.head()


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.93,18.42,61.00,55,0,13.87,ZA,1604179631
1,1,Mataura,-46.19,168.86,57.99,65,20,8.99,NZ,1604179760
2,2,Coyhaique,-45.58,-72.07,77.00,20,0,8.05,CL,1604179761
3,3,Aksarka,66.56,67.80,29.35,97,9,14.38,RU,1604179686
4,4,Roebourne,-20.78,117.13,69.67,78,0,10.42,AU,1604179761


In [3]:
# Clean up presentation of data (remove duplicate index column)

cities_pd = cities_pd.rename(columns={'Unnamed: 0': 'ID'})
cities_pd = cities_pd.set_index('ID')
cities_pd.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
ID,,,,,,,,,
0,Cape Town,-33.93,18.42,61.00,55,0,13.87,ZA,1604179631
1,Mataura,-46.19,168.86,57.99,65,20,8.99,NZ,1604179760
2,Coyhaique,-45.58,-72.07,77.00,20,0,8.05,CL,1604179761
3,Aksarka,66.56,67.80,29.35,97,9,14.38,RU,1604179686
4,Roebourne,-20.78,117.13,69.67,78,0,10.42,AU,1604179761


In [64]:
cities_pd = cities_pd.dropna()
cities_pd.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
ID,,,,,,,,,
0,Cape Town,-33.93,18.42,61.00,55,0,13.87,ZA,1604179631
1,Mataura,-46.19,168.86,57.99,65,20,8.99,NZ,1604179760
2,Coyhaique,-45.58,-72.07,77.00,20,0,8.05,CL,1604179761
3,Aksarka,66.56,67.80,29.35,97,9,14.38,RU,1604179686
4,Roebourne,-20.78,117.13,69.67,78,0,10.42,AU,1604179761


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key = g_key)

In [24]:
## Map & Humidity Heatmap

locations = cities_pd[["Lat", "Lon"]].astype(float)
humidity = cities_pd["Humidity"]

fig_map= gmaps.figure(center = (41.88, 87.62), zoom_level=2.2)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights = humidity,
    dissipating=False,
    max_intensity = 100,
    point_radius = 1)

fig_map.add_layer(heat_layer)
fig_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [116]:
# VACATION NOTES: 
# I'm looking for a warm rainforst-type place - high humidity, high cloud cover, high temp but not too high. 
#  limiting wind factor to a reasonable less than 30 mph to reduce areas experiencing storm conditions. 
# (also remembering that this data is a snapshot, so at least half the max temps represent night temperatures)

#& Drop any cities with NA values

ideal_spots = cities_pd[(cities_pd["Humidity"]>=80)&(cities_pd["Wind Speed"]<30)& (cities_pd["Max Temp"]>75)&
                    (cities_pd["Max Temp"]<85)&(cities_pd["Cloudiness"]>90)].dropna()
ideal_spots = ideal_spots.set_index('City')
ideal_spots

,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,
Bengkulu,-3.80,102.27,75.43,88,100,4.07,ID,1604179692
Avarua,-21.21,-159.78,75.20,83,98,11.41,CK,1604179713
Padang,-0.95,100.35,77.61,80,100,3.69,ID,1604178924
Tabou,4.42,-7.35,78.31,87,100,5.68,CI,1604179796
Trairi,-3.28,-39.27,79.70,81,100,9.71,BR,1604179834
Sungai Raya,0.70,108.90,78.84,83,100,2.04,ID,1604179835
Harper,4.38,-7.72,77.92,88,100,6.11,LR,1604179836
Dingle,11.00,122.67,75.52,84,100,2.77,PH,1604179687
Ranong,9.97,98.63,77.54,82,96,0.63,TH,1604179846


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
#create new dataframe hotel_df with empty column "Hotel Name"
hotel_df = ideal_spots
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.set_index('City')
hotel_df

,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City,,,,,,,,,
Bengkulu,-3.80,102.27,75.43,88,100,4.07,ID,1604179692,
Avarua,-21.21,-159.78,75.20,83,98,11.41,CK,1604179713,
Padang,-0.95,100.35,77.61,80,100,3.69,ID,1604178924,
Tabou,4.42,-7.35,78.31,87,100,5.68,CI,1604179796,
Trairi,-3.28,-39.27,79.70,81,100,9.71,BR,1604179834,
Sungai Raya,0.70,108.90,78.84,83,100,2.04,ID,1604179835,
Harper,4.38,-7.72,77.92,88,100,6.11,LR,1604179836,
Dingle,11.00,122.67,75.52,84,100,2.77,PH,1604179687,
Ranong,9.97,98.63,77.54,82,96,0.63,TH,1604179846,


In [100]:
## create a params dict which will allow it to update with the new city each iteration:

# set params: 5000 meters, Hotels
params = {"key": g_key,
          "radius" : 5000,
          "types" : "lodging",
         }

# Loop through the short list of cities with ideal climates and run a hotel search for each city
for index, row in hotel_df.iterrows():
    
    #set to the google place api
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #set search coordinates to the city's lat/lon found in the hotel list (generated from the 
    #original cities_pd call)
    coord = f"{row['Lat']},{row['Lon']}"

    # update address key value
    params['location'] = coord

    #take it for a loop
    try:
        # make request
        response = requests.get(base_url, params=params)

        # convert to json
        response = response.json()

        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except(KeyError, IndexError):
        print("could not find " + row["City"])
        hotel_df.loc[index, "Hotel Name"] = ["No Hotels"]
        
# Visualize 
hotel_df

could not find Angoram


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
ID,,,,,,,,,,
40,Bengkulu,-3.80,102.27,75.43,88,100,4.07,ID,1604179692,Grage Hotel Bengkulu
69,Avarua,-21.21,-159.78,75.20,83,98,11.41,CK,1604179713,Paradise Inn
100,Padang,-0.95,100.35,77.61,80,100,3.69,ID,1604178924,Mercure Padang
160,Tabou,4.42,-7.35,78.31,87,100,5.68,CI,1604179796,hotêl doufoulougou
293,Trairi,-3.28,-39.27,79.70,81,100,9.71,BR,1604179834,Pousada Field Lilies - Trairi
297,Sungai Raya,0.70,108.90,78.84,83,100,2.04,ID,1604179835,808 artel
299,Harper,4.38,-7.72,77.92,88,100,6.11,LR,1604179836,Screensaver
304,Dingle,11.00,122.67,75.52,84,100,2.77,PH,1604179687,Camp Pasica
338,Ranong,9.97,98.63,77.54,82,96,0.63,TH,1604179846,Tinidee Hotel@Ranong


In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [114]:
# Add marker layer on top of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure

fig_map.add_layer(markers)
fig_map

Figure(layout=FigureLayout(height='420px'))